Language Translation with `nn.Transformer` and torchtext
========================================================

This tutorial shows:

:   -   How to train a translation model from scratch using Transformer.
    -   Use torchtext library to access
        [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)
        dataset to train a German to English translation model.


Data Sourcing and Processing
============================

[torchtext library](https://pytorch.org/text/stable/) has utilities for
creating datasets that can be easily iterated through for the purposes
of creating a language translation model. In this example, we show how
to use torchtext\'s inbuilt datasets, tokenize a raw text sentence,
build vocabulary, and numericalize tokens into tensor. We will use
[Multi30k dataset from torchtext
library](https://pytorch.org/text/stable/datasets.html#multi30k) that
yields a pair of source-target raw sentences.

To access torchtext datasets, please install torchdata following
instructions at <https://github.com/pytorch/data>.


In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List



# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

Create source and target language tokenizer. Make sure to install the
dependencies.

``` {.sourceCode .python}
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```


In [2]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

Seq2Seq Network using Transformer
=================================

Transformer is a Seq2Seq model introduced in ["Attention is all you
need"](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)
paper for solving machine translation tasks. Below, we will create a
Seq2Seq network that uses Transformer. The network consists of three
parts. First part is the embedding layer. This layer converts tensor of
input indices into corresponding tensor of input embeddings. These
embedding are further augmented with positional encodings to provide
position information of input tokens to the model. The second part is
the actual
[Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)
model. Finally, the output of the Transformer model is passed through
linear layer that gives unnormalized probabilities for each token in the
target language.


In [3]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import random
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(p)
        self.embedding = TokenEmbedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_len, N)

        embedding = self.dropout(self.embedding(x))
        # embedding.shape = (seq_len, N , embedding_size)
        _, (hidden,cell) = self.rnn(embedding)

        return hidden, cell

    

class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,num_layers, p):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = TokenEmbedding(vocab_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size,vocab_size)

    def forward(self,x,hidden,cell):
        # x.shape = (N) => (1,N)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        output, (hidden,cell) = self.rnn(embedding, (hidden,cell))

        predictions = self.fc(output)
        # predictions.shape = (1,N,vocab_size)
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell


# Seq2Seq Network
class Seq2SeqLSTM(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqLSTM, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        

    def forward(self, src, tgt):
        batch_size = src.shape[1]
        target_len = tgt.shape[0]
        
        outputs = torch.zeros(target_len,batch_size,TGT_VOCAB_SIZE).to(DEVICE)

        hidden, cell = self.encoder(src)

        x = tgt[0]
        
        for t in range(1,target_len):
            output, hidden, cell = self.decoder(x,hidden,cell)

            outputs[t] = output

            best_guess = output.argmax(1)

            x = best_guess

        return outputs



During training, we need a subsequent word mask that will prevent the
model from looking into the future words when making predictions. We
will also need masks to hide source and target padding tokens. Below,
let\'s define a function that will take care of both.


In [4]:
torch.manual_seed(0)

writer = SummaryWriter(f'runs/loss_plot')

EMB_SIZE = 300
BATCH_SIZE = 128
HIDDEN_SIZE = 512
NUM_LAYERS = 2
DROPOUT = 0.5


encoder = Encoder(SRC_VOCAB_SIZE,EMB_SIZE,HIDDEN_SIZE,NUM_LAYERS,DROPOUT).to(DEVICE)
decoder = Decoder(TGT_VOCAB_SIZE, EMB_SIZE, HIDDEN_SIZE,NUM_LAYERS,DROPOUT).to(DEVICE)

lstm = Seq2SeqLSTM(encoder,decoder).to(DEVICE)

print(lstm)
print("Model parameter count: ",sum(p.numel() for p in lstm.parameters() if p.requires_grad))

for p in lstm.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

Seq2SeqLSTM(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): TokenEmbedding(
      (embedding): Embedding(19214, 300)
    )
    (rnn): LSTM(300, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): TokenEmbedding(
      (embedding): Embedding(10837, 300)
    )
    (rnn): LSTM(300, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=10837, bias=True)
  )
)
Model parameter count:  22111321


Collation
=========

As seen in the `Data Sourcing and Processing` section, our data iterator
yields a pair of raw strings. We need to convert these string pairs into
the batched tensors that can be processed by our `Seq2Seq` network
defined previously. Below we define our collate function that converts a
batch of raw strings into batch tensors that can be fed directly into
our model.


In [5]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Let\'s define training and evaluation loop that will be called for each
epoch.


In [6]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        output = model(src,tgt)
        output = output[1:].reshape(-1,output.shape[2])
        tgt = tgt[1:].reshape(-1)

        optimizer.zero_grad()
        loss = loss_fn(output,tgt)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        output = model(src,tgt)
        output = output[1:].reshape(-1,output.shape[2])
        tgt = tgt[1:].reshape(-1)
        loss = loss_fn(output,tgt)
        losses += loss.item()

    return losses / len(list(val_dataloader))

Now we have all the ingredients to train our model. Let\'s do it!


In [7]:
from timeit import default_timer as timer
NUM_EPOCHS = 30
lstm.to(DEVICE)
minValLoss = float('inf')
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

    # save checkpoint if validation loss improves
    if minValLoss > val_loss:
        print("saving model...")
        minValLoss = val_loss
        bestCheckpoint = f'.data/checkpoints/lstm_v3_epoch={epoch}_valLoss={val_loss}.pth'
        torch.save(lstm.state_dict(), bestCheckpoint)

c:\Users\guy\miniconda3\envs\torch\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 6.023, Val loss: 5.212, Epoch time = 52.376s
saving model...
Epoch: 2, Train loss: 5.089, Val loss: 5.067, Epoch time = 55.117s
saving model...
Epoch: 3, Train loss: 4.978, Val loss: 4.986, Epoch time = 60.844s
saving model...
Epoch: 4, Train loss: 4.881, Val loss: 4.937, Epoch time = 60.761s
saving model...
Epoch: 5, Train loss: 4.803, Val loss: 4.894, Epoch time = 63.374s
saving model...
Epoch: 6, Train loss: 4.737, Val loss: 4.845, Epoch time = 60.977s
saving model...
Epoch: 7, Train loss: 4.681, Val loss: 4.807, Epoch time = 61.987s
saving model...
Epoch: 8, Train loss: 4.629, Val loss: 4.768, Epoch time = 61.641s
saving model...
Epoch: 9, Train loss: 4.583, Val loss: 4.727, Epoch time = 60.399s
saving model...
Epoch: 10, Train loss: 4.541, Val loss: 4.708, Epoch time = 62.043s
saving model...
Epoch: 11, Train loss: 4.496, Val loss: 4.669, Epoch time = 64.602s
saving model...
Epoch: 12, Train loss: 4.452, Val loss: 4.652, Epoch time = 61.962s
saving model...
E

In [9]:
NUM_EPOCHS = 30
for epoch in range(NUM_EPOCHS, NUM_EPOCHS*2+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

    # save checkpoint if validation loss improves
    if minValLoss > val_loss:
        print("saving model...")
        minValLoss = val_loss
        bestCheckpoint = f'.data/checkpoints/lstm_v3_epoch={epoch}_valLoss={val_loss}.pth'
        torch.save(lstm.state_dict(), bestCheckpoint)

c:\Users\guy\miniconda3\envs\torch\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 60, Train loss: 3.637, Val loss: 4.088, Epoch time = 54.383s
saving model...


In [11]:
checkpoint = torch.load(bestCheckpoint)
lstm.load_state_dict(checkpoint)

<All keys matched successfully>

In [12]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, max_len, start_symbol):
    src = src.to(DEVICE)

    hx,cx = model.encoder(src)
    ys = torch.ones(1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for _ in range(max_len-1):
        hx,cx = (hx.to(DEVICE),cx.to(DEVICE))
        out,hx,cx = model.decoder(torch.tensor([ys[-1]]).to(DEVICE),hx,cx)
        _, next_word = torch.max(out, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    tgt_tokens = greedy_decode(
        model,  src, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [13]:
translate(lstm,'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.')

' Two young men are are are the the . . '

In [18]:
translated_sentences = []
references = []
with open('E:\Code\SeminarWork\.data\\test_2016_flickr.de', 'r') as srcs, open('E:\Code\SeminarWork\.data\\test_2016_flickr.en', 'r') as tgts:
    start = timer()
    for src, tgt in zip(srcs, tgts):
        translated_sentence = translate(lstm,src.rstrip())
        translated_sentences.append(translated_sentence)
        references.append(tgt.strip())
    end = timer()

print(f'inference time: {end-start}')

inference time: 15.58476410000003


In [ ]:
from sacrebleu import corpus_bleu

refs = [[ref] for ref in references]

bleu = corpus_bleu(translated_sentences, references)

print(f'BLEU score: {bleu.score:.4f}')


In [ ]:
i = 200
for translated, ref in zip(translated_sentences,refs):
    print(f'translated: {translated}\nreference: {ref}')
    i-=1
    if i == 0:
        break

References
==========

1.  Attention is all you need paper.
    <https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf>
2.  The annotated transformer.
    <https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding>
